# Stanと ~R~ Pythonでベイズ統計モデリング

このノートブックではwaicやwbicの情報量基準をみていく

実装は[このノートブックを参考にした](https://gist.github.com/narrowlyapplicable/580f2d3610190b7bc04693466e409224)

In [6]:
import pystan
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import pickle

plt.style.use("ggplot")

In [7]:
J = 8
y = np.array([28.,  8., -3.,  7., -1.,  1., 18., 12.])
sigma = np.array([15., 10., 16., 11.,  9., 11., 10., 18.])
schools = np.array(['Choate', 'Deerfield', 'Phillips Andover', 'Phillips Exeter',
                    'Hotchkiss', 'Lawrenceville', "St. Paul's", 'Mt. Hermon'])

## model

モデルは０１で使ったモデルをそのまま使う。

In [8]:
schools_model = """
data {
  int<lower=0> J;
  real y[J];
  real<lower=0> sigma[J];
}

parameters {
  real mu;
  real<lower=0> tau;
  real theta[J];
}

model {
  mu ~ normal(0, 5);
  tau ~ cauchy(0, 5);
  theta ~ normal(mu, tau);
  y ~ normal(theta, sigma);
}
generated quantities {
    vector[J] log_lik;
    vector[J] y_hat;
    for (j in 1:J) {
        log_lik[j] = normal_lpdf(y[j] | theta[j], sigma[j]);
        y_hat[j] = normal_rng(theta[j], sigma[j]);
    }
}
"""

stanモデルをコンパイルする。

In [11]:
try:
    with open("./model_code.pkl", "rb") as f:
        stanmodel = pickle.load(f)
except FileNotFoundError:
    stanmodel = pystan.StanModel(model_code=schools_model, model_name="model_code")
    with open("./model_code.pkl", "wb") as f:
        pickle.dump(stanmodel, f)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL model_code_b0eada194ad938e30ed47926216f98ca NOW.


In [10]:
schools_dat = {'J': 8,
               'y': [28,  8, -3,  7, -1,  1, 18, 12],
               'sigma': [15, 10, 16, 11,  9, 11, 10, 18]}

In [15]:
fit = sm.sampling(data=schools_dat, iter=1000, chains=4, warmup=100, seed=496)

### Def waic

ここ実装に自信ないので間違いあったら教えてください。。

渡辺ベイズp１１８参照

waic: $$W_n = T_n + \frac{\beta V_n}{n} \tag{1}$$

経験損失： $$\displaystyle{V_n = \sum_{i=1}^{n} {{E_w}[(log p(X_i|w))^2)] - {E_w}[log p(X_i|w)]^2  }}\tag{2}$$

汎化数分散： $$\displaystyle{T_n = - \frac{1}{n} \sum_{i=1}^{n} log E_w[p(X_i|w)] }\tag{3}$$

[wbicは詳しくは渡辺先生のHPのこのページか](http://watanabe-www.math.dis.titech.ac.jp/users/swatanab/wbic2012.html)
[このページ](http://watanabe-www.math.dis.titech.ac.jp/users/swatanab/waicwbic_e.html*)

$$wbic = E_w^{\frac{1}{log\ n}} \left[\displaystyle{\sum_{i=1}^{n}} log p(X_i|w)\right]$$

In [35]:
def calc_waic(samples, beta=1):
    """
    ==params==
    samples: 対数尤度　log p(X_i|w)
    beta : 逆温度
    ==return==
    waic : 
    """
    Tn = -np.mean(np.log(np.mean(np.exp(samples), axis=0)))
    print("Tn:",Tn)
    Vn = np.sum(np.mean(samples**2, axis=0) - np.mean(samples, axis=0)**2)
    print("Vn",Vn)
    waic = Tn + beta * (Vn/samples.shape[0])
    print("waic", waic)
    return waic

In [47]:
def calc_wbic(samples):
    """
    ==params==
    samples: 対数尤度　log p(X_i|w)
    beta : 逆温度
    ==return==
    wbic :
    """
    beta = 1/np.log(samples.shape[0])
    wbic = - np.mean(beta * np.sum(samples, axis=1))
    print("wbic",wbic)
    return wbic

In [48]:
standata = schools_dat
fit = sm.sampling(data=standata, iter=1000, warmup=300, seed=496)
ms = fit.extract()
waic = calc_waic(ms["log_lik"])
wbic = calc_wbic(ms["log_lik"])

Tn: 3.727941745883408
Vn 0.8458107535515627
waic 3.7282438211525335
wbic 3.803773481546461


WAICやWBICといった情報量基準は、パラメータの事後分布が正規分布で、尤度関数が凸じゃなくても使える。

が、あくまでもモデル間での比較基準に過ぎないことを覚えておきたい。

真のモデルに近いのを選択するには、一致性をもつWBICを用いるらしい。。（まだわかってない。）

次はモデリング時の基準となる情報量基準について、実装を確認したので、次はモデリング部分についてやっていきたい